In [1]:
#set the API key
from tmdbv3api import TMDb
from tmdbv3api import Movie
tmdb = TMDb()
movie = Movie()
tmdb.api_key = 'c6cd98753f10dc390a8c4a5b4b83d47c'
tmdb.language = 'en'
tmdb.debug = True

In [11]:
import requests

In [6]:
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
response = requests.get("https://www.themoviedb.org/movie", headers = needed_headers) 


In [7]:
response.status_code


200

In [8]:
dwn_content = response.text
len(dwn_content)

205992

In [34]:
dwn_content[:500]
'<!DOCTYPE html>\n<html lang="en" class="no-js">\n  <head>\n    <title>Popular TV Shows &#8212; The Movie Database (TMDB)</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <meta http-equiv="cleartype" content="on">\n    <meta charset="utf-8">\n    \n    <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">\n    <meta name="mobile-web-app-capable" content="yes">\n    <meta name="'

'<!DOCTYPE html>\n<html lang="en" class="no-js">\n  <head>\n    <title>Popular TV Shows &#8212; The Movie Database (TMDB)</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n    <meta http-equiv="cleartype" content="on">\n    <meta charset="utf-8">\n    \n    <meta name="keywords" content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast">\n    <meta name="mobile-web-app-capable" content="yes">\n    <meta name="'

In [13]:
from bs4 import BeautifulSoup

In [15]:
test_doc = BeautifulSoup(response.text, 'html.parser')
type(test_doc)

bs4.BeautifulSoup

In [18]:
test_doc.find('title')

<title>Popular Movies — The Movie Database (TMDB)</title>

In [19]:
def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page

In [20]:
popular_shows_url = "https://www.themoviedb.org/movie"
doc = get_page_content(popular_shows_url)

In [21]:
doc.title.text


'Popular Movies — The Movie Database (TMDB)'

In [22]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.text


'Black Adam'

In [23]:
doc.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']


'73.0'

In [24]:
def empty_dict():
    scraped_dict = {  
    'Title': [],
    'User_rating': [], 
    'Release_date':[], 
    'Current_season': [],
    'Current_season_Episodes': [], 
    'Tagline': [],
    'Genre': [],
    'Cast': []   
    }
    return scraped_dict

In [25]:
def user_score_info(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])

In [35]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.a['href']


'/movie/436270'

In [26]:
def get_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'})
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        # here we are creating the list of all the individual pages of the shows which will come handy in other functions. 
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
       # we are going to have the function to return the list of all the information as elements. 
    return tag_title, tag_user_score, doc_2_list
# lets see if the function returns the list of the information we tried to get earlier. 
len(get_show_info(doc))

3

In [30]:
doc 2 = get page content("https://www.themoviedb.org/tv/91363")

SyntaxError: invalid syntax (655109888.py, line 1)

In [27]:
tag_genre = doc_2.find('span', {"class": "genres"})
tag_genre_list = tag_genre.find_all('a')
check_genre =[]
for tag in tag_genre_list:
    check_genre.append(tag.text)
check_genre
['Animation', 'Action & Adventure', 'Sci-Fi & Fantasy']

NameError: name 'doc_2' is not defined

In [28]:
# lets create a function to get the genres for the show. 
# i here denotes the element of the list vairable ``doc2_page`` that contains different doc pages. Will come handy later on.
def get_genres(doc2_page, i):
    genres_tags = doc2_page[i].find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre

In [29]:
tag_tagline = doc_2.find('h3',{"class": 'tagline'})

def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'].append(doc_2_list[i].find('h3',{"class": 'tagline'}).text)
    else:
        scraped_dict['Tagline'].append("No Tagline")

NameError: name 'doc_2' is not defined

In [31]:
import pandas as pd

def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_dict =  empty_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'].append(t_title[i].h2.text)
        user_score_info(t_user_score, i, scraped_dict)    
        scraped_dict['Release_date'].append(t_title[i].p.text)
        scraped_dict['Current_season'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h2.text)
        scraped_dict['Current_season_Episodes'].append(docs_2_list[i].find_all('div' , {'class': 'flex'})[1].h4.text[7:])
        tagline_info(docs_2_list, i, scraped_dict)  
        scraped_dict['Genre'].append(get_genres(docs_2_list, i))        
        scraped_dict['Cast'].append(get_show_cast(docs_2_list, i))
        
    return pd.DataFrame(scraped_dict)


In [32]:
x = get_show_details(tag_title_, tag_user_score_, doc_2_list_)
x.to_csv('check.csv')
pd.read_csv('check.csv',index_col=[0])

NameError: name 'tag_title_' is not defined

In [33]:
movie = Movie()
popular = movie.popular()
for p in popular: 
    print(p.id) 
    print(p.title) 
    print(p.overview) 
    print(p.poster_path)

436270
Black Adam
Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.
/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg
736526
Troll
Deep inside the mountain of Dovre, something gigantic awakens after being trapped for a thousand years. Destroying everything in its path, the creature is fast approaching the capital of Norway. But how do you stop something you thought only existed in Norwegian folklore?
/9z4jRr43JdtU66P0iy8h18OyLql.jpg
724495
The Woman King
The story of the Agojie, the all-female unit of warriors who protected the African Kingdom of Dahomey in the 1800s with skills and a fierceness unlike anything the world has ever seen, and General Nanisca as she trains the next generation of recruits and readies them for battle against an enemy determined to destroy their way of life.
/lQMJHnHxUyj8kJlC2YOKNuzuwMP.jpg
76600
Avatar